#INSTALLS

In [1]:
!pip install vaderSentiment-fr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.1/187.1 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 8.4 MB/s eta 0:00:00
  Created wheel for vaderSentiment-fr: filename=vaderSentiment_fr-1.3.4-py3-none-any.whl size=185960 sha256=ec34bb799c266481caeb451cdab7cc5e2948a8fcd668b5431283b118a2bc1adf
  Stored in directory: /root/.cache/pip/wheels/a3/a0/97/307bbd5f16d2e240c69421a79a42a18797067af832ee955b09
Successfully built vaderSentiment-fr


In [2]:
!pip install textblob_fr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.2/561.2 kB 5.1 MB/s eta 0:00:00


#IMPORTS

In [3]:
import re
import numpy as np
import random
import pandas as pd
import nltk
from nltk.corpus import stopwords
from tqdm import tqdm
import os
import nltk
from vaderSentiment_fr.vaderSentiment import SentimentIntensityAnalyzer

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer
tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())

#DIRECTORY

In [5]:
# mount my Google Drive on the VM

from google.colab import drive
drive.mount('/gdrive')

DIR_PROJECT = '/gdrive/MyDrive/Projet5ADD/Tous les Codes'
path = os.path.join(DIR_PROJECT,'Code Sentiment Analysis/French-NRC-EmoLex.txt')


Mounted at /gdrive


#RESSOURCES

In [6]:
# Download NLTK resources (stopwords and punkt tokenizer)
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

https://saifmohammad.com/WebPages/NRC-Emotion-Lexicon.htm

In [7]:
# Load NRC Emotion Lexicon
lexicon = pd.read_csv(path, sep='\t')

In [8]:
lexicon.head()

,English Word,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust,French Word
0,aback,0,0,0,0,0,0,0,0,0,0,surpris
1,abacus,0,0,0,0,0,0,0,0,0,1,abaque
2,abandon,0,0,0,1,0,1,0,1,0,0,abandonner
3,abandoned,1,0,0,1,0,1,0,1,0,0,abandonné
4,abandonment,1,0,0,1,0,1,0,1,1,0,abandon


#VADER

In [14]:
# Create an instance of the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Define a function to perform sentiment analysis using VADER
def get_sentiment(tweet):
    sentiment = analyzer.polarity_scores(tweet)
    compound_score = sentiment['compound']
    return compound_score

# Define a function to categorize sentiment based on VADER score
def categorize_sentiment(score):
    if score < 0:
        return 'negative'
    elif score > 0:
        return 'positive'
    else:
        return 'neutral'

# Define a function to perform emotion detection using the lexicon
def detect_emotion(tweet):

    emotions = {'anger': 0, 'anticipation': 0, 'disgust': 0, 'fear': 0, 'joy': 0, 'sadness': 0, 'surprise': 0, 'trust': 0}

    words = tweet.lower().split()
    emotion_col = set(emotions.keys())
    for word in words:
        matches = lexicon[(lexicon['French Word'] == word)]
        if not matches.empty:
          for emotion in emotion_col:
            emotions[emotion] += matches[emotion].iloc[0]

    # Check if all emotion values are still zero
    if all(value == 0 for value in emotions.values()):
        return None

    # Determine the predominant emotion
    predominant_emotion = max(emotions, key=emotions.get)

    return predominant_emotion

# Example sentence
example_sentence = "Je suis loin d'être heureux"

# Apply sentiment analysis to the example sentence
sentiment_score = get_sentiment(example_sentence)
sentiment_category = categorize_sentiment(sentiment_score)

# Apply emotion detection to the example sentence
emotion = detect_emotion(example_sentence)

# Print results
print(f"Sentence: {example_sentence}")
print(f"Sentiment Score: {sentiment_score}")
print(f"Sentiment Category: {sentiment_category}")
print(f"Detected Emotion: {emotion}")


Sentence: Je suis loin d'être heureux
Sentiment Score: 0.5719
Sentiment Category: positive
Detected Emotion: anticipation


#TEXTBLOB

In [15]:
tb(example_sentence).sentiment[0]

0.7